In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip "/content/drive/MyDrive/dataset.zip" -d "/content/"

unzip:  cannot find or open /content/drive/MyDrive/dataset.zip, /content/drive/MyDrive/dataset.zip.zip or /content/drive/MyDrive/dataset.zip.ZIP.


In [ ]:
import os #control folders
import shutil #copy paste folders
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model

from sklearn.metrics import classification_report

import gradio as gr


In [ ]:
'''
train_dir = "/content/chest_xray/train"
val_dir   = "/content/chest_xray/val"
test_dir  = "/content/chest_xray/test"
'''

In [ ]:
'''
def check_and_clean(folder):
    bad_dir = folder + "_bad_images"
    os.makedirs(bad_dir, exist_ok=True)
    for cls in os.listdir(folder):
        class_path = os.path.join(folder, cls)
        for img in os.listdir(class_path):
            img_path = os.path.join(class_path, img)
            try:
                Image.open(img_path).verify()
            except:
                shutil.move(img_path, bad_dir)

check_and_clean(train_dir)
check_and_clean(val_dir)
check_and_clean(test_dir)
'''

In [ ]:
'''
IMG_SIZE = (224, 224)
BATCH = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.3,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
).flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    color_mode="grayscale"
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    color_mode="grayscale"
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary",
    color_mode="grayscale",
    shuffle=False
)
'''

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
'''
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    factor=0.3, #dec learning rate by 0.3
    min_lr=1e-6 #not less than 1e-6
)

class SaveEvery5Epochs(Callback):
    def __init__(self, save_path):
        super().__init__() #callback instructor
        self.save_path = save_path
        os.makedirs(save_path, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None): #logs=dictionary epoch
        if (epoch + 1) % 5 == 0:
            filename = f"model_epoch_{epoch+1}.h5"
            filepath = os.path.join(self.save_path, filename)
            self.model.save(filepath)
            print(f"\n✔️ Saved model at epoch {epoch+1} → {filepath}")

save_callback = SaveEvery5Epochs("saved_models")
'''

In [ ]:
'''
# CNN Simple Model
model_cnn = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,1)),
    layers.MaxPooling2D(), #reduce image size
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(), #convert feature map to 1 vector
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_cnn.summary()
'''

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,168,513 (42.60 MB)

 Trainable params: 11,168,513 (42.60 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
'''
# Deep CNN Model
def build_deep_cnn(input_shape=(224,224,1)):
    model = models.Sequential([
        layers.Conv2D(16, (1,1), activation='relu', input_shape=input_shape),
        layers.Conv2D(32, (3,3), padding='same', activation='relu'),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(32, (1,1), activation='relu'),
        layers.Conv2D(64, (3,3), padding='same', activation='relu'),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (1,1), activation='relu'),
        layers.Conv2D(128, (3,3), padding='same', activation='relu'),
        layers.MaxPooling2D((2,2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5), #prevent overfitting
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model_deepcnn = build_deep_cnn()
model_deepcnn.summary()
'''

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 16)   │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 32)   │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 64)     │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,947,553 (49.39 MB)

 Trainable params: 12,947,553 (49.39 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
'''
# Training CNNs
# ===========================
history_cnn = model_cnn.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[early_stop, reduce_lr, save_callback]
)

history_deepcnn = model_deepcnn.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[early_stop, reduce_lr, save_callback]
)
'''

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 647s 4s/step - accuracy: 0.7534 - loss: 0.6360 - val_accuracy: 0.8125 - val_loss: 0.5091 - learning_rate: 0.0010
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 658s 4s/step - accuracy: 0.8583 - loss: 0.3292 - val_accuracy: 0.6875 - val_loss: 0.7548 - learning_rate: 0.0010
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 652s 4s/step - accuracy: 0.8674 - loss: 0.3068 - val_accuracy: 0.6875 - val_loss: 0.7953 - learning_rate: 0.0010
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 646s 4s/step - accuracy: 0.8819 - loss: 0.2665 - val_accuracy: 0.7500 - val_loss: 0.5195 - learning_rate: 0.0010
Epoch 5/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9052 - loss: 0.2300


✔️ Saved model at epoch 5 → saved_models/model_epoch_5.h5
163/163 ━━━━━━━━━━━━━━━━━━━━ 658s 4s/step - accuracy: 0.9052 - loss: 0.2300 - val_accuracy: 0.6250 - val_loss: 1.0874 - learning_rate: 3.0000e-04
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 653s 4s/step - accuracy: 0.9172 - loss: 0.2226 - val_accuracy: 0.6250 - val_loss: 0.7177 - learning_rate: 3.0000e-04
Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1030s 6s/step - accuracy: 0.7603 - loss: 0.5064 - val_accuracy: 0.5625 - val_loss: 1.2913 - learning_rate: 0.0010
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 990s 6s/step - accuracy: 0.8204 - loss: 0.3644 - val_accuracy: 0.6250 - val_loss: 0.6510 - learning_rate: 0.0010
Epoch 3/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 976s 6s/step - accuracy: 0.8562 - loss: 0.3357 - val_accuracy: 0.7500 - val_loss: 0.5193 - learning_rate: 0.0010
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 985s 6s/step - accuracy: 0.8630 - loss: 0.3015 - val_accuracy: 0.7500 - val_loss: 0.4506 - learning_rate: 0.0010
Epoch 5/50
163/163 ━━━━━━


✔️ Saved model at epoch 5 → saved_models/model_epoch_5.h5
163/163 ━━━━━━━━━━━━━━━━━━━━ 972s 6s/step - accuracy: 0.8861 - loss: 0.2879 - val_accuracy: 0.6250 - val_loss: 0.7350 - learning_rate: 0.0010
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1034s 6s/step - accuracy: 0.8955 - loss: 0.2609 - val_accuracy: 0.6875 - val_loss: 0.7798 - learning_rate: 0.0010
Epoch 7/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 988s 6s/step - accuracy: 0.9029 - loss: 0.2412 - val_accuracy: 0.6250 - val_loss: 0.7715 - learning_rate: 0.0010
Epoch 8/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1016s 6s/step - accuracy: 0.9024 - loss: 0.2214 - val_accuracy: 0.6250 - val_loss: 0.7623 - learning_rate: 3.0000e-04
Epoch 9/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1007s 6s/step - accuracy: 0.9081 - loss: 0.2160 - val_accuracy: 0.6250 - val_loss: 0.9265 - learning_rate: 3.0000e-04


In [ ]:
'''
#Save final models
model_cnn.save("saved_models/cnn_final.h5")
model_deepcnn.save("saved_models/deepcnn_final.h5")
'''

In [ ]:
IMG_SIZE = (224, 224)

model_cnn = load_model("/content/drive/MyDrive/saved_models/cnn_final.h5")
model_deepcnn = load_model("/content/drive/MyDrive/saved_models/deepcnn_final.h5")

def predict_image(img):
    img = img.convert("L").resize(IMG_SIZE)
    img_array = img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)  # batch dimension

    # CNN
    pred_cnn = model_cnn.predict(img_array)
    pred_cnn_label = "Pneumonia" if pred_cnn[0][0] > 0.5 else "Normal"

    # Deep CNN
    pred_deepcnn = model_deepcnn.predict(img_array)
    pred_deepcnn_label = "Pneumonia" if pred_deepcnn[0][0] > 0.5 else "Normal"

    return pred_cnn_label, pred_deepcnn_label


In [ ]:
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(label="CNN Prediction"),
        gr.Label(label="Deep CNN Prediction")
    ],
    title="X-Ray Pneumonia Classifier",
    description="Upload a Chest X-Ray image to get predictions from CNN and Deep CNN"
)

iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d431377ae1d1cba7a9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
